In [13]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import qmc
from scipy.stats import norm

Grundsätzliche Parameter festlegen

In [161]:
T = 1
d = 2
delta_t = T/d

S_0 = 100
mu = 0.045
sigma = 0.3


Standard Brownian Motion Simulation

In [162]:
def create_standard_bm(d, delta_t):
#create lower trinangular matrix with ones
    lower_tri_matr = np.tril(np.ones([d,d]))
    A = math.sqrt(delta_t)*lower_tri_matr
    #create Brownian motion
    w = A.dot(np.random.normal(0,1,d))

    return w
    #plt.plot(w)
    #plt.show()


Asset

In [163]:
def create_asset_prices(d, delta_t, S_0, mu, sigma):
    S = np.zeros(d+1) #+1 weil d mal simuliert wird und aber der Startpreis des Assets auch in dem Vektor steht
    S[0] = S_0
    delta_S = []

    w = create_standard_bm(d, delta_t)

    #erstelle gesamten Vekjtor w mit w_0
    delta_w = np.concatenate((w[0],np.diff(w)),axis=None)

    for i in range(d):
        delta_S.append(mu*S[i]*delta_t + sigma*S[i]*delta_w[i])
        S[i+1] = S[i] + delta_S[i]

    return S
    
    '''
    plt.plot(s)
    plt.title("Asset prices for mu = " + str(mu) + ", S_0 = " + str(s_0) + ", Sigma = " + str(sigma))
    plt.ylabel("Asset prices")
    plt.show()
    '''


Test, ob Asset prices richtig modelliert sind. Vergleich mit exakter Berechnung ergibt, dass Ergebnis stimmt.

In [164]:
number_paths = 10000
mc_vector = np.empty(number_paths)
for i in range(number_paths):
    mc_vector[i] = create_asset_prices(d, delta_t, S_0, mu, sigma)[2]
display(np.average(mc_vector))

display(S_0*math.exp(mu*d*delta_t))

104.03143789097915

104.6027859908717

In [165]:
#WICHTIG: S_0 muss im Vektor S sein!!! (per Definition)
def ratched_option(d, delta_t, S_0, mu, sigma): 
    S = create_asset_prices(d, delta_t, S_0, mu, sigma)
    #display(S)
    #indic = np.array([1 if s > 0 else 0 for s in np.diff(S)])
    indic = np.diff(S)>0
    #display(indic)
    payoff = np.average(indic*S[1:(S.size)])  #python spezifisch fehlt dann das erste Element
    display(S)
    display(S[1:(S.size)])
    return payoff

In [160]:
ratched_option(d, delta_t, S_0, mu, sigma)

array([100.        ,  53.11320996,  60.84631441])

array([53.11320996, 60.84631441])

30.42315720324412

# Monte Carlo Simulation

Exaktes Ergebnis

In [166]:
Exp = (S_0/d)*norm.cdf((mu-((sigma**2)/2))*math.sqrt(delta_t)/sigma+sigma*math.sqrt(delta_t))*np.sum(np.exp(mu*delta_t*(np.array(range(d))+1)))
display(Exp)

60.40825215088241

Monte Carlo Simulation

In [167]:
number_paths = 1000
mc_vector = np.empty(number_paths)
for i in range(number_paths):
    mc_vector[i] = ratched_option(d, delta_t, S_0, mu, sigma)
np.average(mc_vector)

array([100.        , 118.13044634, 118.07192781])

array([118.13044634, 118.07192781])

array([100.        ,  94.16971803, 120.40878517])

array([ 94.16971803, 120.40878517])

array([100.        ,  99.03677473, 121.30583026])

...

array([100.        ,  98.97329936, 117.91761013])

array([ 98.97329936, 117.91761013])

array([100.        , 107.61567556,  74.92486162])

array([107.61567556,  74.92486162])

63.29472008102475